In [21]:
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFacePipeline
from dotenv import load_dotenv  
import os

from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain_core.output_parsers import StrOutputParser


load_dotenv()
keys = os.getenv("HUGGING_FACE_TOKEN")

llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    model_kwargs={
        'temperature': 0.7,
        "max_length": 400,
        "do_sample": True,
        "pad_token_id": 50256
    }
)

book_prompt = PromptTemplate(
    input_variables=["query"],
    template="You are a book expert. Answer the following question about books: {query}",
    output_parser=RouterOutputParser()
)

book_chain = book_prompt | llm | StrOutputParser()

electronics_prompt = PromptTemplate(
    input_variables=["query"],
    template="You are an electronics expert. Answer the following question about electronics: {query}",
    output_parser=RouterOutputParser()
)

electronics_chain = electronics_prompt | llm | StrOutputParser()


default_prompt = PromptTemplate(
    input_variables=["input"],
    template="Answer this general question: {input}"
)
general_chain = default_prompt | llm | StrOutputParser()


Device set to use cpu


Creating a router

In [22]:
from langchain_core.runnables import RunnableBranch

routing_chain = RunnableBranch(
    # Route to electronics chain if electronics keywords are found
    (lambda x: any(word in x["input"].lower() for word in 
     ["electronic", "circuit", "voltage", "current", "resistor", "capacitor", 
      "transistor", "computer", "hardware", "electrical", "technology"]), 
     electronics_chain),
    
    # Route to book chain if book keywords are found
    (lambda x: any(word in x["input"].lower() for word in 
     ["book", "author", "novel", "literature", "reading", "story", "chapter", 
      "fiction", "recommend", "library", "write", "wrote"]), 
     book_chain),
    
    # Default to general chain
    general_chain
)

In [ ]:
test_questions = [
    "What is the best way to learn circuit analysis?",
    "Can you recommend a good fantasy book?",
    "What's the weather like today?",
    "How do resistors work in electronic circuits?",
    "Who wrote the book 1984?",
    "What is voltage in electrical circuits?",
    "What are some good science fiction novels?"
]

print("Testing the routing system using | operator:\n")
for question in test_questions:
    print(f"Question: {question}")
    try:
        result = routing_chain.invoke({"query": question})
        print(f"Answer: {result}")
        print("-" * 50)
    except Exception as e:
        print(f"Error: {e}")
        print("-" * 50)

Testing the routing system using | operator:

Question: What is the best way to learn circuit analysis?
Error: 'input'
--------------------------------------------------
Question: Can you recommend a good fantasy book?
Error: 'input'
--------------------------------------------------
Question: What's the weather like today?
Error: 'input'
--------------------------------------------------
Question: How do resistors work in electronic circuits?
Error: 'input'
--------------------------------------------------
Question: Who wrote the book 1984?
Error: 'input'
--------------------------------------------------
Question: What is voltage in electrical circuits?
Error: 'input'
--------------------------------------------------
Question: What are some good science fiction novels?
Error: 'input'
--------------------------------------------------


: 